<a href="https://colab.research.google.com/github/cassiecinzori/ECON3916/blob/main/Labs/Lecture9/The_Architecture_of_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Architecture of Control

### Cassandra Cinzori

## Step 1: Environment and Data Strategy

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from scipy import stats


df = pd.read_csv('lalonde_data.csv')
print(df.head())
print(df['treat'].value_counts())

     ID  treat  age  educ  black  hispan  married  nodegree  re74  re75  \
0  NSW1      1   37    11      1       0        1         1   0.0   0.0   
1  NSW2      1   22     9      0       1        0         1   0.0   0.0   
2  NSW3      1   30    12      1       0        0         0   0.0   0.0   
3  NSW4      1   27    11      1       0        0         1   0.0   0.0   
4  NSW5      1   33     8      1       0        0         1   0.0   0.0   

         re78  
0   9930.0460  
1   3595.8940  
2  24909.4500  
3   7506.1460  
4    289.7899  
treat
0    429
1    185
Name: count, dtype: int64


## Step 2: The Observational Failure (Naive Comparison)

In [8]:
naive_diff = (df[df.treat == 1]['re78'].mean() -
              df[df.treat == 0]['re78'].mean())
print(f"Naive Difference in Means: ${naive_diff:,.2f}")
# Expected: ~-$635.03  ← selection bias at work

Naive Difference in Means: $-635.03


## Step 3: Propensity Score Estimation

In [11]:
# Confounders: variables that affect BOTH treatment selection AND earnings
covariates = ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75']


X = df[covariates]
y = df['treat']

logit = LogisticRegression(solver='liblinear', max_iter=1000)
logit.fit(X, y)

df['pscore'] = logit.predict_proba(X)[:, 1]

print("\nPropensity score summary:")
print(df.groupby('treat')['pscore'].describe())


Propensity score summary:
       count      mean       std       min       25%       50%       75%  \
treat                                                                      
0      429.0  0.196656  0.226224  0.009029  0.053966  0.107163  0.168812   
1      185.0  0.561869  0.211763  0.040171  0.475299  0.669336  0.706804   

            max  
treat            
0      0.778098  
1      0.755915  


## Step 4: The Matching Algorithm (Nearest Neighbor)

In [12]:
treated = df[df.treat == 1].copy()
control = df[df.treat == 0].copy()

# Fit NN on control propensity scores
nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
nbrs.fit(control[['pscore']])

# Find the closest control unit for each treated unit
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()].copy()

# Construct matched dataset
matched_df = pd.concat([treated, matched_control]).reset_index(drop=True)

print(f"\nMatched sample size: {len(matched_df)} "
      f"({len(treated)} treated + {len(matched_control)} matched controls)")


Matched sample size: 370 (185 treated + 185 matched controls)


## Step 5: Assessing Balance and Estimating the Effect

In [13]:
def smd(var, df):
    t = df[df.treat == 1][var]
    c = df[df.treat == 0][var]
    pooled_sd = np.sqrt((t.var() + c.var()) / 2)
    return (t.mean() - c.mean()) / pooled_sd if pooled_sd > 0 else 0

print("\nBalance Check (SMD < 0.1 is good):")
print(f"{'Covariate':<12} {'Before':>8} {'After':>8}")
print("-" * 30)
for var in covariates:
    before = smd(var, df)
    after  = smd(var, matched_df)
    flag   = " ✓" if abs(after) < 0.1 else " ⚠"
    print(f"{var:<12} {before:>8.3f} {after:>8.3f}{flag}")


diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"\nRaw Effect (Difference):          ${diff:,.2f}")
print(f"P-value:                           {p_val:.4f}")


matched_treated = matched_df[matched_df.treat == 1]['re78']
matched_control_out = matched_df[matched_df.treat == 0]['re78']

matched_diff = matched_treated.mean() - matched_control_out.mean()
t_stat_m, p_val_m = stats.ttest_ind(matched_treated, matched_control_out)

print(f"\nRecovered Effect (Matched Diff):  ${matched_diff:,.2f}")
print(f"P-value:                           {p_val_m:.4f}")
# Target: ~$1,500–$1,800 range, recovering the true experimental benchmark


Balance Check (SMD < 0.1 is good):
Covariate      Before    After
------------------------------
age            -0.242    0.261 ⚠
educ            0.045    0.027 ✓
black           1.668    0.015 ✓
hispan         -0.277    0.129 ⚠
married        -0.719   -0.131 ⚠
nodegree        0.235    0.206 ⚠
re74           -0.596    0.049 ✓
re75           -0.287    0.080 ✓

Raw Effect (Difference):          $-635.03
P-value:                           0.3342

Recovered Effect (Matched Diff):  $583.04
P-value:                           0.4438
